In [1]:
import os
import pandas as pd
import geopandas as gpd
from utils import *

In [2]:
STRATCASE_SHP_DIR = "/Users/tszchun.chow/Library/CloudStorage/OneDrive-SharedLibraries-AucklandTransport/Congestion Charging - General/Lead Advisor (shared folder)/02 Analysis (Transport Modelling)/Reporting and Documentation/Strategic Case Graphics/Graphics/data"

In [3]:
network = gpd.read_parquet('/Users/tszchun.chow/Documents/GitHub/nz-abm-core/tabular-outputs/outputs/ToUCH_DoMin_2026/monty-tables/links.parquet').set_crs('epsg:4326').to_crs('epsg:2193')

In [4]:
sector_3 = MSM_ZONES.dissolve('Sector_3')
lba = MSM_ZONES.dissolve('LBA_Name')

In [5]:
sa2_2023 = gpd.read_file(os.path.join(STRATCASE_SHP_DIR, 'statistical-area-2-2023-generalised.gpkg'))
sa2_2023_centroid = sa2_2023.copy()
sa2_2023_centroid['geometry'] = sa2_2023_centroid.centroid

In [6]:
akl_sa2_2023_centroid = sa2_2023_centroid.sjoin(LOCAL_BOARD_AREAS)[['SA22023_V1_00', 'LBA_2', 'geometry']]
akl_sa2_2023_centroid = akl_sa2_2023_centroid.sjoin_nearest(sector_3[['Sector_1', 'Sector_2', 'geometry']]).drop(columns='geometry')
akl_sa2_2023 = akl_sa2_2023_centroid.merge(sa2_2023[['SA22023_V1_00','SA22023_V1_00_NAME', 'geometry']], on='SA22023_V1_00')
akl_sa2_2023 = gpd.GeoDataFrame(akl_sa2_2023, geometry=akl_sa2_2023['geometry'], crs='epsg:2193')
# akl_sa2_2023.drop(columns='geometry').to_csv(os.path.join(STRATCASE_SHP_DIR, 'SA22023_lba_sector_crosswalk.csv'), index=None)
# akl_sa2_2023.to_file(os.path.join(STRATCASE_SHP_DIR, 'SA22023.shp'))

In [7]:
akl_region = akl_sa2_2023.dissolve()[['geometry']]

In [8]:
akl_network = network.sjoin(akl_region)
akl_network.to_file(os.path.join(STRATCASE_SHP_DIR, 'AKL_network.shp'))

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_79505/1736908707.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  akl_network.to_file(os.path.join(STRATCASE_SHP_DIR, 'AKL_network.shp'))
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'monty_link_class' to 'monty_link'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'allowed_modes' to 'allowed_mo'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'freespeed_kmh' to 'freespeed_'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-

# POPULATION (2023)

In [49]:
cordon_1a = gpd.read_file(os.path.join(STRATCASE_SHP_DIR, 'Options', '1A_CBD_Cordon.geojson'))[['geometry']].to_crs('epsg:2193')
cordon_1c = gpd.read_file(os.path.join(STRATCASE_SHP_DIR, 'Options', '1C_City_Fringe.geojson'))[['geometry']].to_crs('epsg:2193')
area_2 = gpd.read_file(os.path.join(STRATCASE_SHP_DIR, 'Options', '2_Inner_Isthmus_Area.geojson'))[['geometry']].to_crs('epsg:2193')

In [50]:
population_2023 = pd.read_csv(os.path.join(STRATCASE_SHP_DIR, 'population_census_2023.csv'))

# rc = usual residents
rc_2023 = population_2023[population_2023['CEN23_TBT_IND_003']=='rc'][['CEN23_TBT_GEO_006', 'Area', 'OBS_VALUE']].rename(columns={'OBS_VALUE': 'usual_residents'})
rc_2023_gdf = sa2_2023.merge(rc_2023, left_on = 'SA22023_V1_00', right_on='CEN23_TBT_GEO_006')

In [54]:
def option_population_overlap(rc_2023_gdf, option_area):
    opt_sa2_2023 = rc_2023_gdf.overlay(option_area, how='intersection')# .explore()
    opt_sa2_2023['overlap_area'] = opt_sa2_2023.area/1000000
    opt_sa2_2023['overlap_percentage'] = (opt_sa2_2023['overlap_area']/opt_sa2_2023['AREA_SQ_KM']).round(3)
    opt_sa2_2023['overlap_percentage'] = opt_sa2_2023['overlap_percentage'].apply(lambda x: x if x < 0.99 else 1)
    opt_sa2_2023['overlap_residents'] = opt_sa2_2023['usual_residents']*opt_sa2_2023['overlap_percentage']
    return opt_sa2_2023[['SA22023_V1_00', 'SA22023_V1_00_NAME', 'usual_residents', 'overlap_residents']]

opt1a_population = option_population_overlap(rc_2023_gdf, cordon_1a)
opt1a_population['option'] = '1A'
opt1c_population = option_population_overlap(rc_2023_gdf, cordon_1c)
opt1c_population['option'] = '1C'
opt2a_population = option_population_overlap(rc_2023_gdf, area_2)
opt2a_population['option'] = '2A'

option_populations = pd.concat([opt1a_population, opt1c_population, opt2a_population])
option_populations.to_excel(os.path.join(STRATCASE_SHP_DIR, 'Options', 'Option_Population_2023.xlsx'), index=None)

In [17]:
LOCAL_BOARD_AREAS

,OBJECTID,TALB2021_V1_00,TALB2021_V1_00_NAME,TALB2021_V1_00_NAME_ASCII,LAND_AREA_SQ_KM,AREA_SQ_KM,Shape_Length,LBA_Sector,geometry,Local Board Area,LBA_2
0,27,07601,Rodney Local Board Area,Rodney Local Board Area,2275.114945,5809.371481,450393.401621,Other Areas,"MULTIPOLYGON (((1788533.265 6047342.8, 1789163...",Rodney,Rodney
1,28,07602,Hibiscus and Bays Local Board Area,Hibiscus and Bays Local Board Area,110.097301,454.636873,93891.587670,Hibiscus and Bays,"MULTIPOLYGON (((1752023.352 5954803.281, 17520...",Hibiscus and Bays,Hibiscus and Bays
2,29,07606,Henderson-Massey Local Board Area,Henderson-Massey Local Board Area,53.222381,64.610913,47647.307032,West Auckland,"MULTIPOLYGON (((1742041.111 5926852.531, 17422...",Henderson-Massey,Henderson - Massey
3,30,07607,Waitākere Ranges Local Board Area,Waitakere Ranges Local Board Area,305.757350,943.010242,148727.154193,West Auckland,"MULTIPOLYGON (((1732107.57 5922583.663, 173212...",Waitākere Ranges,Waitakere Ranges
4,31,07605,Devonport-Takapuna Local Board Area,Devonport-Takapuna Local Board Area,20.044551,55.820572,34573.930619,North Shore,"MULTIPOLYGON (((1755483.96 5931760.622, 175552...",Devonport-Takapuna,Devonport - Takapuna
5,32,07608,Aotea/Great Barrier Local Board Area,Aotea/Great Barrier Local Board Area,320.268863,4485.550194,285021.355854,Other Areas,"MULTIPOLYGON (((1814479.116 6033535.274, 18148...",Aotea/Great Barrier,Aotea/Great Barrier
6,73,07603,Upper Harbour Local Board Area,Upper Harbour Local Board Area,69.725954,82.761361,49132.446541,North Shore,"MULTIPOLYGON (((1754053.06 5936385.066, 175407...",Upper Harbour,Upper Harbour
7,74,07612,Albert-Eden Local Board Area,Albert-Eden Local Board Area,28.343561,33.000712,39592.284578,Inner Isthmus,"MULTIPOLYGON (((1752353.662 5919429.471, 17524...",Albert-Eden,Albert - Eden
8,75,07604,Kaipātiki Local Board Area,Kaipatiki Local Board Area,34.068191,49.171284,28733.207132,North Shore,"MULTIPOLYGON (((1755063.799 5930218.865, 17551...",Kaipātiki,Kaipatiki
9,76,07609,Waiheke Local Board Area,Waiheke Local Board Area,154.758966,1353.463758,156893.166797,Other Areas,"MULTIPOLYGON (((1804349.466 5928406.497, 18060...",Waiheke,Waiheke


In [13]:
sa1 = gpd.read_file(os.path.join(STRATCASE_SHP_DIR, 'statistical-area-1-2021-generalised.gpkg')).sort_values('SA12021_V1_00')
deprivation_index = pd.read_excel(os.path.join(STRATCASE_SHP_DIR, 'nzdep2018-statistical-area-1-sa1-data-730395.xlsx'))
deprivation_index
sa1['SA12021_V1_00'] = sa1['SA12021_V1_00'].astype(int)
deprivation_index['SA12018_code'] = deprivation_index['SA12018_code'].astype(int)

In [23]:
deprivation_index_gdf = sa1[['SA12021_V1_00', 'geometry']].merge(deprivation_index, left_on='SA12021_V1_00', right_on='SA12018_code')

In [4]:
sa2 = gpd.read_file(os.path.join(STRATCASE_SHP_DIR, 'statistical-area-2-2018-generalised.geojson'))
geo_crosswalk = pd.read_csv(os.path.join(STRATCASE_SHP_DIR, 'geographic-areas-table-2022.csv'))

In [86]:
sa1_regc_map = geo_crosswalk[['SA12022_code', 'SA22022_code', 'REGC2022_name']]
akl_sa1_list = sa1_regc_map[sa1_regc_map['REGC2022_name']=='Auckland Region'][['SA12022_code']].drop_duplicates()
akl_dep_index_gdf = deprivation_index_gdf.merge(akl_sa1_list, left_on='SA12018_code', right_on='SA12022_code')[['SA12018_code', 'NZDep2018', 'URPopnSA1_2018', 'geometry']]
akl_dep_index_gdf.to_file(os.path.join(STRATCASE_SHP_DIR, 'deprivation_index_Auckland.shp'))

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_39873/3986431088.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  akl_dep_index_gdf.to_file(os.path.join(STRATCASE_SHP_DIR, 'deprivation_index_Auckland.shp'))
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'SA12018_code' to 'SA12018_co'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'URPopnSA1_2018' to 'URPopnSA1_'
  ogr_write(


In [24]:
akl_sa2 = sa2[['SA22018_V1_00', 'name', 'geometry']].rename(columns={'SA22018_V1_00': 'SA22018_code'})
akl_sa2_list = geo_crosswalk[geo_crosswalk['REGC2022_name'].str.contains('Auckland')][['SA22018_code']].drop_duplicates()
akl_sa2 = akl_sa2.merge(akl_sa2_list, on='SA22018_code')
akl_sa2 = akl_sa2[~akl_sa2['name'].str.contains("Oceanic|Inlet|Bays Waiheke")]
akl_sa2.to_file(os.path.join(STRATCASE_SHP_DIR, 'Auckland_SA2.geojson'))

In [126]:
msm_lba_sector_crosswalk = pd.read_csv('../inputs/msm_zones/msmzone_lba_sector_crosswalk.csv').drop(columns=['Shape_Leng', 'Shape_Area'])
msmzones_gdf = gpd.read_file('../inputs/msm_zones/MSM2018_zones.shp')
sector_3 = msmzones_gdf.merge(msm_lba_sector_crosswalk, on='MSM2018').dissolve(['Sector_3'])
sector_1 = msmzones_gdf.merge(msm_lba_sector_crosswalk, on='MSM2018').dissolve(['Sector_1']).reset_index()
sector_1.to_file(os.path.join(STRATCASE_SHP_DIR, 'Sectors.shp'))
sector_3.to_file(os.path.join(STRATCASE_SHP_DIR, 'Sectors_3.shp'))

/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 538627459.743999958 of field Shape_Area of feature 5 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 379525397.777999997 of field Shape_Area of feature 8 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 538627459.743999958 of field Shape_Area of feature 22 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3

In [66]:
akl_sa2['centroid'] = akl_sa2.centroid
akl_sa2_centroids = gpd.GeoDataFrame(akl_sa2, geometry = akl_sa2['centroid'], crs='epsg:2193')[['SA22018_code', 'geometry']]
sa2_sector_lookup = akl_sa2_centroids.sjoin_nearest(sector_3)[['SA22018_code', 'LBA_Name', 'Sector_2', 'Sector_3']].drop_duplicates()
sa2_sector_lookup.to_csv('./sa2_lba_sector_lookup.csv', index=None)
# akl_sa2.merge(sa2_sector_lookup, on='SA22018_code').drop(columns='centroid')

In [79]:
akl = akl_sa2.dissolve()

In [80]:
akl_lba = gpd.read_file(os.path.join(STRATCASE_SHP_DIR, 'akl_lba.geojson')).to_crs('epsg:2193')
akl_lba['intersection'] = akl_lba.apply(lambda x: x['geometry'].intersection(akl['geometry']), axis=1)
akl_lba_exact = akl_lba[['TALB2021_V1_00_NAME', 'intersection']].rename(columns={'intersection': 'geometry'})# .plot()
akl_lba_exact = gpd.GeoDataFrame(akl_lba_exact, geometry='geometry', crs='epsg:2193')
akl_lba_exact['name'] = akl_lba_exact['TALB2021_V1_00_NAME'].str.replace(" Local Board Area", "")
akl_lba_exact.to_file(os.path.join(STRATCASE_SHP_DIR, 'Auckland_LocalBoardAreas.gpkg'))

In [97]:
akl_lba_exact['TALB2021_V1_00_NAME'] = akl_lba_exact['TALB2021_V1_00_NAME'].str.replace('ā', 'a').str.replace('ē', 'e').str.replace('ī', 'i').str.replace('ō', 'o').str.replace('ū', 'u').str.replace('Ō', 'O')

In [124]:
household_income = pd.read_csv(os.path.join(STRATCASE_SHP_DIR, 'household_income.csv'))
household_income = household_income[household_income['Household income']=='Median income']
household_income_lba = household_income[(household_income['YEAR_CEN18_FHH_022']==2018)&(household_income['AREA_CEN18_FHH_022'].str.contains('CMB'))][['AREA_CEN18_FHH_022', 'Area', 'OBS_VALUE']]
household_income_lba['Area'] = household_income_lba['Area'].str.replace("Great Barrier", "Aotea/Great Barrier")
household_income_sa2 = household_income[(household_income['YEAR_CEN18_FHH_022']==2018)&(household_income['AREA_CEN18_FHH_022'].str[0]=='1')
                                        ][['AREA_CEN18_FHH_022', 'Area', 'OBS_VALUE']].rename(columns={'AREA_CEN18_FHH_022': 'sa2_code'})
household_income_sa2['sa2_code']= household_income_sa2['sa2_code'].astype(int)
household_income_sa2 = akl_sa2.drop(columns='centroid').merge(household_income_sa2, left_on='SA22018_code', right_on='sa2_code')
household_income_sa2.to_file(os.path.join(STRATCASE_SHP_DIR, 'household_income_sa2.shp'))
# lba_MHAI_gdf = akl_lba_exact.merge(household_income_lba, right_on = 'Area', left_on='TALB2021_V1_00_NAME').to_crs('epsg:4326')
# lba_MHAI_gdf.to_file(os.path.join(STRATCASE_SHP_DIR, 'household_income_LBA.shp'))

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_39873/501411628.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  household_income_sa2.to_file(os.path.join(STRATCASE_SHP_DIR, 'household_income_sa2.shp'))
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'SA22018_code' to 'SA22018_co'
  ogr_write(


In [79]:
SECTOR_MAP = {'CBD': 'CBD',
              'City Fringe East': 'City Fringe',
                'City Fringe West': 'City Fringe',
                'East Auckland': 'East Auckland',
                'Franklin': 'Franklin',
                'Hibiscus Coast': 'Hibiscus Coast',
                'Inner Isthmus Central': 'Inner Isthmus',
                'Inner Isthmus West': 'Inner Isthmus',
                'Isthmus Central': 'Outer Isthmus West',
                'Isthmus East': 'Outer Isthmus East',
                'Isthmus West': 'Outer Isthmus West',
                'North Shore': 'North Shore',
                'Other Areas': 'Other Areas',
                'Rodney': 'Rodney',
                'South Auckland': 'South Auckland',
                'West Auckland': 'West Auckland'
}

In [84]:
msmzones = gpd.read_file(os.path.join(STRATCASE_SHP_DIR, 'MSM_Zones.shp'))
sectors = msmzones.dissolve(by='Sector_1').reset_index()[['Sector_1', 'geometry']]
sectors['Sector'] = sectors['Sector_1'].map(SECTOR_MAP)
sectors.dissolve(by='Sector').reset_index()[['Sector', 'geometry']].to_file(
    os.path.join(STRATCASE_SHP_DIR, 'Sectors.gpkg')
)